In [22]:
import math
import numpy as np

In [23]:
import sys

sys.setrecursionlimit(10**6) 


In [24]:
import requests

# reads an uri and returns its content split by line, deals with \n and \r
def read_uri(uri):
  response = requests.get(uri)
  content = response.text.strip().replace("\r", "").split("\n")
  return content

In [25]:
def dp_change(money, coins):
  # [0  1  2  3  4  5  6  7  8  9 10]
  # [1 0 0 0 0 0 0 0 0 0 0]
  # [0 1 0 1 0 0 0 0 0 0 0]
  # [0 0 2 0 2 0 2 0 0 0 0]
  # [0 0 0 3] i don't care about this situation
  # [0 0 0 0 4]
  possibles = [math.inf] * (money+1)
  possibles[0] = 1
  # assuming its always possible as if there is always a 1 coin denomination
  for j in range(len(possibles)):
    if possibles[j]:
      for coin in coins:
        if j + coin <= money:
          possibles[j + coin] = min(possibles[j] + 1, possibles[j + coin])
  return possibles[money] - 1

In [26]:
print(dp_change(40,[50,25,20,10,5,1]))

2


In [27]:
dp_change(17028, [14,5,3,1]) # 1218

1218

In [28]:
def list_string_to_ints(s):
  return list(map(int, s.split()))

def read_manhattan(uri):
  content = read_uri(uri)
  n, m = list(map(int, content[0].split()))
  down = np.array(list(map(list_string_to_ints, content[1:n+1])))
  right = np.array(list(map(list_string_to_ints, content[n+2:2*n+3])))
  print(down, right)
  return n, m, down, right

In [29]:
read_manhattan("https://raw.githubusercontent.com/guilhermesilveira/bioinformatics/main/datasets/manhattan.txt")

[[1 0 2 4 3]
 [4 6 5 2 1]
 [4 4 5 2 1]
 [5 6 8 5 3]] [[3 2 4 0]
 [3 2 4 2]
 [0 7 3 3]
 [3 3 0 2]
 [1 3 2 2]]


(4, 4, array([[1, 0, 2, 4, 3],
        [4, 6, 5, 2, 1],
        [4, 4, 5, 2, 1],
        [5, 6, 8, 5, 3]]), array([[3, 2, 4, 0],
        [3, 2, 4, 2],
        [0, 7, 3, 3],
        [3, 3, 0, 2],
        [1, 3, 2, 2]]))

In [30]:
def longest_path(n, m, down, right):

  # print(n, m, len(down), len(right))

  maximal = [0] * (m+1)
  for j in range(1, m + 1):
    maximal[j] = maximal[j - 1] + right[0][j - 1]

  # print(maximal)

  for i in range(1, n+1):
    new_maximal = [0] * (m + 1)
    new_maximal[0] = maximal[0] + down[i - 1][0]
    for j in range(1, m+1):
      up = maximal[j] + down[i - 1][j]
      left = new_maximal[j-1] + right[i][j - 1]
      new_maximal[j] = max(up, left)
    maximal = new_maximal
  return maximal[m]

In [31]:
longest_path(*read_manhattan("https://raw.githubusercontent.com/guilhermesilveira/bioinformatics/main/datasets/manhattan.txt"))

[[1 0 2 4 3]
 [4 6 5 2 1]
 [4 4 5 2 1]
 [5 6 8 5 3]] [[3 2 4 0]
 [3 2 4 2]
 [0 7 3 3]
 [3 3 0 2]
 [1 3 2 2]]


34

In [32]:
longest_path(*read_manhattan("https://raw.githubusercontent.com/guilhermesilveira/bioinformatics/main/datasets/dataset_261_10.txt"))

[[4 3 3 1 3]
 [0 2 3 0 4]
 [3 1 2 2 4]
 [4 4 2 0 0]
 [1 2 1 4 0]
 [1 1 0 4 1]
 [1 0 1 2 0]
 [4 1 1 3 0]
 [4 2 0 1 3]
 [1 0 2 0 1]] [[3 4 0 0]
 [3 1 4 3]
 [4 1 1 1]
 [0 3 3 2]
 [4 4 1 0]
 [0 0 1 4]
 [3 0 2 3]
 [0 2 4 1]
 [3 1 4 3]
 [3 3 4 3]
 [2 1 4 2]]


40

In [33]:
def lcs(v, w):
  lv = len(v)
  lw = len(w)
  s = np.zeros((lv + 1, lw + 1))
  backtrack = np.zeros(s.shape)
  for i in range(1, lv + 1):
    for j in range(1, lw + 1):
      matches = v[i-1] == w[j-1]
      s[i, j] = max(s[i-1,j], s[i,j-1], s[i-1,j-1] + matches)
      if s[i,j] == s[i-1,j]:
          backtrack[i, j] = 1
      elif s[i, j] == s[i, j-1]:
          backtrack[i, j] = 2
      else:
          backtrack[i, j] = 0
      # print(i, j, matches, v[i-1], w[j-1])
      # print(s, backtrack)
  return s, backtrack

def lcs_output(backtrack, v, i, j):
  # print(i, j)
  if i == 0 or j == 0:
    return ""
  if backtrack[i, j] == 1:
      return lcs_output(backtrack, v, i - 1, j)
  elif backtrack[i, j] == 2:
      return lcs_output(backtrack, v, i, j - 1)
  else:
      return lcs_output(backtrack, v, i - 1, j - 1) + v[i-1]

In [34]:
s, bt = lcs("AACCTTGG", "ACACTGTGA")
print(bt)
lcs_output(bt, "AACCTTGG", len("AACCTTGG"), len("ACACTGTGA"))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 2. 2. 2. 2. 2. 2. 2.]
 [0. 1. 1. 0. 2. 2. 2. 2. 2. 2.]
 [0. 1. 0. 1. 0. 2. 2. 2. 2. 2.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 0. 2. 2. 2. 2.]
 [0. 1. 1. 1. 1. 1. 1. 0. 2. 2.]
 [0. 1. 1. 1. 1. 1. 0. 1. 0. 2.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


'AACTTG'

In [35]:
s, bt = lcs("AA", "AC")
print(s)
print(bt)
lcs_output(bt, "AA", len("AA"), len("AC"))

[[0. 0. 0.]
 [0. 1. 1.]
 [0. 1. 1.]]
[[0. 0. 0.]
 [0. 0. 2.]
 [0. 1. 1.]]


'A'

In [36]:
s1,s2 = "CTTTTACGAGATACTCAGCCACTTCAATCCTGAGGATCGACAGGGTTATTGGCAGACAGGGCAATGCCGTTTTGCCCTGGATCGTACTGGCTTCCCCATTTATACCTCAGACATTTCATGAGTTGTCGACTCTCTCATACTATGTGCAAATTTATCCCCACGATGGGAACGATTACATATACTGGGCAGTGGATTGCGGTAAAATCGGCTGATCTGCAATGATGATTTGAAATTTCGTCTGCCCGGGATCGGTCTCGAAGAGAATTCATAATGCTTAAGCCTATACTTACAGCTATAGTGAGTCAATGAGTTACCTTAGGCTGCCCGTTGGGAGTCTTCTTCATCCTAACAGGCACGTAGCCTTCGAAATCTTTGATCCCTCAACCCTTATGCTAAGCGAAGAATGTTTCAACACGTTTGGTCTTACTCTGCGCTTTGCAGAAGTGGACCTTACGCCTATATCTGACGTACTCCAGTGCAATATTTCCGGGTAGCCCGGCTAAGTATTCACAGACAAAGTAGGCTTATGGATGCATACAAGAGTCCACCGATCGGTCATTGTATGAAGCAAACTTGTGCGTCAAAGTTCATGAATAGTCACATAATGCAAATGCACCAAAGCATCTCGTAGCAATGTCTGATGAGTAGAATAGGGCGAGGGAGGGGCGCACCTACGGTTTCTAGTCAATGCAGTACAGAAGGAGGAATATGAGGACTCCTTGAGAGAGATGGTAAGGGTCTTTAGACTGAAGTATCTTTTGAAACCGCCATCGGCAGGTTATTCGCTACCGGAAGGGCGACGTAGAAACGTAGTTTCATGGGTAGTAACTGCATAAATAATCGTTCTAATAACCTCTGGAGATTACATTTCACTCCATGAATTGGTGCTGATAGGTGTGTAACCCCAGCAGAGGGTG", "GACTACGATCAAGTCAGGTCCCAGAGCCTAGCATTGTCCCTTTTAGCAGCTGCGAATAATAGACCGCTTGCCTTTATCTCCCCCCGTACGGGGCCTAGTGCAATGAAAACTAGGCTTAAGGACCCTCAGTAAACACCGGTGTCATATTACACGGGCTCAAGTCGGCTTGTCAGCCAAAAAGTACTGGGACTCGAGGCTGCTCTCAATCTAGCAACGTCGCTTCGCGATATTTACGAAATTCGTCCTAGTCACATGATGGCAAGCCATAAGCGTCGAGTTAAAACTACCACGCTTGTCTAGCGCCATTGCTGTCCATGATACATACAAGGTAGTTCCATTGCATTGACTGAATAGAACAGTCAACGGATTCTTTTGCGACCGGCGGACCTAGAGCATAGGAGGGAGTGTATCCAGAAGCCACCGCGTTCAGGTAGTGGTGAGGTTGCTATTACAAGCATGTTCGGTGGGAGGAGGATTGTTTCCACGAAACCACTCGCCCCTCAAGCCCCGGGAACGATGACTAGGTGGATCTAGACAGTCAGGGTAATCAGAGTACTGAAATACGGAACAAATAGAGGGGGACAATCATGGCGGCGGGAGAGCGTAGCTTACAGGTACCAGTTCACGGTTATTTCCAACATGGTGGCGCTAAAACTCACTCGCCTTGATTCTGTAGCTGCCGTCGCAAAACTGAACCGTTGCGAGATCAGTTATGGCGCATGAAATGTTGCATATGTTTATAAGCCGTCAACTGCAAGTCAAGACCAATTGTTACTTCCCCTCTGCGAGTCGACTCAAACAGAACAGTTCGCAGAGATAACCGTACAAGGCCGGATATTGGCACCCGGTTAAGCCATTGATGGGACTTATAAACGTCCTCTAACGCCACCCCCGCATACTGAATATTTTAAAGGGCGTTGACAACATCGAACGACCTGCTTTAGATAGCATTGATCTGGTGCTTGCATTTTTTTAA"
print(len(s1), len(s2))
s, bt = lcs(s1, s2)
lcs_output(bt, s1, len(s1), len(s2))

917 976


'CTACGATATCAGCCCAACCTAGATGCTTTTGCAGCGGAATGCCGTTGCCTATCGTACGGCTTCAATACTAGCTTAAGGACCCTCATACATGTCAATTACCCCAGTGGCGTACAATACTGGGCAGTGATTGCAATCGCTTCGCGATATTTGAAATTCGTCTGCCGATGGCAAGAAATCATAACTAAGCTTCTACGCATGTGTCATGATACTACGGTAGTCTTCTTACTAAAGACAGTCAAATCTTTGACCCACCTAGCTAAGGAGTGTTCAAAGCACCGCGTTCAGAGTGGATTGCTATACGAGTTCGTGAAATTTCCGGTAGCCCGGCTAAGTATTAGACAGTAGGTAATCAGAGTACGATCGGCATGAGACAAATGGCGCAAAGTTCATAAGTCACATAATGCAAATCACCGCATCTGTAGCTGTCGATGAGTAGAATAGGGCAGAGGCATAGTTTTAGTCAATGCAGTCAAGAAATTGACTCCTTGGAGGATAACAGACTGAAGATAACCGCAGGCGGTATTGCACCGGAAGGGGACTAAAACGTTCTAGACGCATAAATATTTTAATAACTCGAGATCTTTATCATGATTGGTGCTGATTTTAA'

In [37]:
s1,s2 = "ACCGTCTTAGCGATCAACACATTTAACAACGCGCCGCACCCCCCGTCAAACGAGCTTTTGGGCTCTTGTCCTTTTACAAGCTTCACGACGCATACAGCCTTGATCAACGGTTTGATCTGTCTCCCTTCAGCTGGCTTTAAAGGACATACATATGAAGGCCTTAATAAGGTCCGGGAACTCCACATATTCGGTACTGGGCAAACCCCATGAACCACCTCAACATGAAGAGTCCGAGGACTCTCACGATCCACCAATGCAGATCGGAACTGTGCGATCGCGTAATGAGCCGAGTACTTGGTTTGTGTTTAGGTTATGGGGGCCGGGAGCCGGTTCAATATAAGGAAGTAGTTGCAGATTAGTTGTTGCGAACGGTCATAAATTTGATGGGTAAACGTGAACTTAACAAACCGTGATAGCTAATCCTATGCATCCCTTACGTGGATCGACTCGAGTACCCAGGTGAACCGACTACTTGATAACCGGAAATCGCGGTATAAAAGCGCTCACGGTCAGGAGATATACCTCCAAGCAGTAGTCTTTCTGAGCCTAGAGTAGTAAATTACAGGGACGATGTCTTTTACCGAGGCAACATTTTATTGAGAATCACATGAGGCACAGGTAAAGGCGACATCACGATCGAGATCAACCCCTACTTGTTCAAAACATTGAGAACCAGCTCTGTTTTGGAACCTAGAAAGATAACGCATCCGCTTGATATTCCACGGCTTGTCCCTCTTGTGCGGTCCATCTATCGGAGTTTCCTCCGATACGACCCGCAATGTTTCCAGGCGTACGGTACTTTATGAATACACTCGCGCTGTAACCTGTTATGTGAAACACACACGACAGAGCTTCGCGTGGGCCCAGCGACCCGGTAATACTACATCACCGCACACGACCTCGAGCAGTCTTTGCCGGCGTCCGTAAGTAGTCTAAAGTTGTGTTGATGCTTGGGGTTAAAGCTAAATCGTCCGCAGAATACGACTCTCATCCCAAT","ACCCGCACGCGCTTTGGTCTAGATTCTAGCTCCAACTTGCCTGCTAGATACTCTGTTAAAAGATGGTTTTACAACCCCCTCCTCTGTCCCTGGGGTATTATATAATACGTCGGATAGTCAGGTACAAATACAAGTGGGTGGGAATACTTTTCCTCGGATCCTAGACCACGGATTACTGCGTGGTTGACAAGAGTCGGCCCGGAGGGAAACGTGAAGGTTAGTGCAATTAAAGTCTCTAATGTGAAGCCTCCGCGAAGCGAGGAGTTTCTGAGATCGAGTACTATTTAGAGTTCGAAATCACGGCTTAACCTCACTGCCACGCATAACTTGCCGGCAATCCAGTTTTGCAACGATACTTAATTTGTGCAGCTCATCTTTGCTGTCCAGAAATAGAGCTAGTCGATCTCATCTTGCGGGTAGCCAGAAGTCCTACCGTCTCCTCCATGTAGCTTAAAAATTTCGGTGAGGATCAAAAATGATAAACGTGACAGGTAAGCTCCTACGTCTATCCTATGACCCCCGCGGCAGAATAGGTTGGTAGTGTTAGTGCGTGAGCTGGTAGAATAGAGCACACTTAGGGAAACGGGAACCGTTATGTAGGGCTGCGACACACAAAAAAGTGTTCGTTGGTAAGCTGCCTCTCCACTAAACAGGATTTCTCTGGATGATCCCATCGAAGCAAGTTACGCACCACGCCGAGGCGGACCCTGGTACTAGCTGCCCCCCCCTTTATGGGGCGCTCGTACATCAAGATGATCGCGGACTCAACCTGATTACGAGTTGTCCAAGTAGTCCAGGGTAAGAGAAACTGGAGAGA"
s, bt = lcs(s1, s2)
len(lcs_output(bt, s1, len(s1), len(s2)))

573

In [38]:
len("ACCGCAGCGTCAATTTACAACGCCGCACCGTAAAGATGGTTTTACAACCCCCTCCCTGTCCGGTTTATTTCTCTAGTCAGGACAAATAAAGTGGTGGGAATACTTTCTCGGACCAGACCACTACTGGTGGTTGACAAGAGTCGGCCCGGAGGGAACTGGTTGTGTTAGTTATGGGCCCCGGAAGGAGAGTTGAGATCGAGTCTATTTGAGTCGAATCACGGCTAACCTATGCACCTACTTGCCGATCCAGTGAACGATACTTATACCATCGCGTAAAAAGGCTAGTCGATATCCTCCAGAGTAGTCTTCTGAGCTAAAAATTCGGGAGATCAAAAATATAAACTGACAGGTAAGCCTACGTCATCAACCCCCGCAAAATTGGAGTGTTTTGGCTAGAAAGAGCACCTTGAAACGGGCCTTTGTGGGTCCACACAGTTTCTGTAAGCTGTTCCACTACGGTCTTATGATCATCGGCAAGTTAGCACCACGCGAGGCGGACCCGGTACTACTCCCCCACGCTCGACATCTTGCGGCTCCTGATTAAGTTGTGTGTCGGGTAAAGAAACTGAGAGA")

573

In [39]:
s1, s2 = "GACT","ATG"
s, bt = lcs(s1, s2)
r = lcs_output(bt, s1, len(s1), len(s2))
print(len(r), r)

2 AT


In [40]:
s1, s2 = "ATG", "GACT"
s, bt = lcs(s1, s2)
print(s, bt)
r = lcs_output(bt, s1, len(s1), len(s2))
print(len(r), r)

[[0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1.]
 [0. 0. 1. 1. 2.]
 [0. 1. 1. 1. 2.]] [[0. 0. 0. 0. 0.]
 [0. 1. 0. 2. 2.]
 [0. 1. 1. 1. 0.]
 [0. 0. 1. 1. 1.]]
2 AT


In [41]:
s1, s2 = "AGAGTAGACACCTTGCGCTCTGTTTGATATGGAAGCTCGCAGGTCCAGAGTGAAATCATAGAAATTACCATAAGCGATGATATGAAGCACAGATCACTGTTTGTGCTCAGTGGCATCGACTCTATTTAAACATAGTATACCGGTTTATTATAGGTCCTGTGCCTGGCCTCGTGTGCCGTGCCTTGGGATTTGAACGCTGAACTACCCACGCGCACGGAAGGGGTTCCAGTGCGCTAACAATAACACTGTATCCCAGTCGAGTCTGATCGCCCTAGAAAAGCAATTGCACTCAGGGAAACGTGCGGCGGCTCGGTCATCTCGATTGACGTGGGGTCCGCCGCTGAATTTAAAGACAGGCTATGATTGTTCCTTAACCGATCAAGCATGCGATTATTGGTCCCCTATACTCTATTACGTAATGCGGTGATCTGGTACTGGCAGCTGTCGATCTCGTGAATCTCCCGCTGCATTATTCGCCGTACTACCGAATGAGAGTGATCCCCGCCGGGCACTGCACCTGTGCGCCGGAGAATCGAGTTCATCACAACCACTGCTACAGACCTCTATAGACCGTAGACCGACCCCACGGTCCGTTATGAAGGAAGTCCAAAAGAGGCGGATGTCCTACGCTCTCTTCGCAGATCTATTCATACCTCCGGGTAAATTCTATGACAGCCCTACCACGGGATATGTACACTCTCACATGGAGATTTTTAGGTACAGATATTCATTGATGTACAATGGGCGGAGCGCCAGCGGACCGACCGAAATATCGAGTGTCGAGGATCAATCACGTTGCTACGTGCACCCGGCACCCGACAAATTCACGTGCAGCCGTCTCTTGGTAAGCGGGCACCGTCACGGATCGTGCGTGCCTAGCCTAGTGCCCGTGGGGCTATTGCTGGGGTATCCCCTTGGGAGAAGTTACGACGTTGATGATCCC", "CAGGAAGTGGTAGATAACCACTAGCTGACCAAGCAACCAGCATTCATACAACTTATTTAGGGAAGAGAAGGAGCAAGGTGTGGAGAAGTCATACCCCGTATGGATACGTTATTTGGGTCCAGAAATTAGTATAGGTGCCAACGAACAATTTCGATTCAATTAGATAGCGGTTCGCACCTCCTTCGGAGTGTGTGCGCAGCAGCTATCTTGCCCGCAGAAAGAACGCGGTGTCCCTGGGCTAACCGGGATGGTAAGCATTTTATAATCTGTGCACACGTCTATCATGAGACCTCTTCATGCCCGGTCGAAGGCGTAGGGATCAGCGGCTGGGATGGCCAAGTGCACGCCTTTTATGAGGGGATCCACTCTACTTGCGATCGCAACGTGCTGGGGTACGGCAACGTCCCCAGTAGGCTCGTTGCTGTACACGACTGCAGGTACTTGTTGAGGCAGGGCGCCCAATATAGTGGTCTGGACTCCGGCCCCATGATATTGATTATCACGTCATCGTGCCAACCTGAAACGGACTTTAGCTAGATGCTGGTCTGGTAGAATCAGAGATCTCCTTGTCCGTCTGTAAATCAGTGCAATCACGGAAATGTTGACACTATGTATGATAATCTGTTTCGAAACCGAACACGGTACACGGCTACGTACACTCGGTGTGTACATCAGCCTCAGCGCCCGAATCTCTGCGAAGATCAATCCTTGACCTCGGTGATACTTCGCTTAAATGTCGCGATACACAGCGTTCATAGAATAAGTAAACATACGCTGTAAGCTGGATAAGAAGAATTTTAGAAACTCAA"
s, bt = lcs(s1, s2)
r = lcs_output(bt, s1, len(s1), len(s2))
print(len(r), r)

568 AGAGTAGACACTGCGCCAAGAACCGCATCATAAATATAGAAAAAAGCGTGTAGAAGCACAGATACGTTTTTGGGCAGATTATTTAAACAATTCGATTAATTGTGCGGTCGCCTCCTTGGGTTTGCGCGACTACCCCGCGAGAAGGGGTCCTGGCTAACATAACATTATCCCAGTCATCTGAGCCCTCATGCCCGGGAACGTGGGCGGCTGGATCCAAGTGCCGCCTTTTAGAGGGATCCTTACCGATCAAGCTGCGAAGTCCCCATACTCGTTGCGTACGACTGCAGGTACTTGTTGGCAGCGCCCAATAAGTGTCGGACTCCGGCCCAGAATGATTATCACCACTGCCAACCTCATAGCTAGAGCGGTCGTAGAATCAGAGATTCCTGTCTCTTAATCATCAATCCGGAAATTTGACACTAGTATGTAATCTCAAAGAAGGTACAGATACATTGTGTACATCAGCCCAGCGCCCGAATTCTGCGAGATCAATCCTTGCTCGTGAACCGCAAATTCCGTCAGCGTTCTGTAAGTACATCGCTGTAAGCTGGTAGGATTTAGAAACCAA


In [42]:
r

'AGAGTAGACACTGCGCCAAGAACCGCATCATAAATATAGAAAAAAGCGTGTAGAAGCACAGATACGTTTTTGGGCAGATTATTTAAACAATTCGATTAATTGTGCGGTCGCCTCCTTGGGTTTGCGCGACTACCCCGCGAGAAGGGGTCCTGGCTAACATAACATTATCCCAGTCATCTGAGCCCTCATGCCCGGGAACGTGGGCGGCTGGATCCAAGTGCCGCCTTTTAGAGGGATCCTTACCGATCAAGCTGCGAAGTCCCCATACTCGTTGCGTACGACTGCAGGTACTTGTTGGCAGCGCCCAATAAGTGTCGGACTCCGGCCCAGAATGATTATCACCACTGCCAACCTCATAGCTAGAGCGGTCGTAGAATCAGAGATTCCTGTCTCTTAATCATCAATCCGGAAATTTGACACTAGTATGTAATCTCAAAGAAGGTACAGATACATTGTGTACATCAGCCCAGCGCCCGAATTCTGCGAGATCAATCCTTGCTCGTGAACCGCAAATTCCGTCAGCGTTCTGTAAGTACATCGCTGTAAGCTGGTAGGATTTAGAAACCAA'

In [300]:
class Path:
  def __init__(self, path):
    self.path = path

  def __repr__(self):
    return "->".join(map(str,self.path))

In [301]:
from collections import defaultdict

class Graph:

  def __init__(self):
    self.adjacency = defaultdict(dict)
    self.all_nodes = set()

  def connect(self, a, b, w):
    self.adjacency[a][b] = w
    self.all_nodes.add(a)
    self.all_nodes.add(b)

  def __topological_dfs(self, current, stack, visited):
    if current in visited:
      return stack
    visited.add(current)
    neighboors = self.adjacency[current].keys()
    for node in neighboors:
      self.__topological_dfs(node, stack, visited)
    stack.append(current)
    return stack

  # unfortunately, there is no need for all N to be on the [0,...,N-1] interval :(  
  def __max_nodes_required(self):
    return (max(self.all_nodes) + 1)


  def components(self):
    components = [0] * self.__max_nodes_required()
    for i in self.all_nodes:
      components[i] = i
    # can be implemented faster with references, this is O(n^3)
    for i in self.all_nodes:
      for j in self.adjacency[i].keys():
        for k in self.all_nodes:
          if components[k] == j:
            components[k] = components[i]
    return components

  def component_containing(self, key):
    components = self.components()
    component = {i for i in self.all_nodes if components[i] == components[key]}
    return component

  # DFS implementation, nasty...
  def longest_path(self, starting, ending):
    component = self.component_containing(starting)
    topological = self.topological()
    topological_on_path = [i for i in topological if i in component]
    max_distance = [0] * self.__max_nodes_required()
    previous = [-1] * self.__max_nodes_required()
    appeared = False
    for a in topological:
      # print(a, component, appeared)
      if a == starting:
        appeared = True
      if a in component and appeared:
        for b, w in self.adjacency[a].items():
          new_distance = max_distance[a] + w
          if new_distance > max_distance[b]:
            max_distance[b] = new_distance
            previous[b] = a
    path = [ending]
    current = ending
    while current != starting:
      current = previous[current]
      path.append(current)
    path = list(reversed(path))
    return max_distance[ending], Path(path)


  def topological(self):
    stack = []
    visited = set()
    for i in self.all_nodes:
      self.__topological_dfs(i, stack, visited)
    return list(reversed(stack))

  def __repr__(self):
    return f"{self.all_nodes}:::{self.adjacency.__repr__()}"

In [302]:
def adjancecy_string_to_graph(adj_string: str):
  graph = Graph()
  for line in adj_string.split("\n"):
    a = int(line.split("->")[0])
    b = int(line.split("->")[1].split(":")[0])
    w = int(line.split(":")[1])
    graph.connect(a, b, w)
  return graph

In [303]:
g = adjancecy_string_to_graph("""0->1:7
0->2:4
2->3:2
1->4:1
3->4:3
5->6:10""")
g

{0, 1, 2, 3, 4, 5, 6}:::defaultdict(<class 'dict'>, {0: {1: 7, 2: 4}, 2: {3: 2}, 1: {4: 1}, 3: {4: 3}, 5: {6: 10}})

In [304]:
g.topological()

[5, 6, 0, 2, 3, 1, 4]

In [305]:
g.component_containing(0)

{0, 1, 2, 3, 4}

In [306]:
g.longest_path(0, 4)

(9, 0->2->3->4)

In [307]:
g = adjancecy_string_to_graph("""0->6:11
6->12:11
12->30:1
30->49:10
7->47:3
15->42:19
48->49:20
42->49:9
23->43:1
18->42:4
37->42:2
38->47:2
28->41:15
18->33:10
0->42:6
1->49:16
16->31:18
32->35:4
26->44:10
7->40:11
25->28:15
34->41:1
31->42:5
25->42:17
17->34:10
22->27:13
17->22:16
19->35:15
24->35:18
36->47:19
25->49:18
15->29:15
17->21:6
10->33:6
0->47:14
5->20:2
7->43:4
40->41:13
12->35:16
30->42:4
16->39:2
24->48:2
23->31:7
16->20:4
20->29:19
46->49:4
5->28:12
25->37:7
35->41:11
45->47:1
14->15:14
0->35:7
11->32:12
36->41:12
3->27:11
29->40:17
22->33:8
19->23:20
21->30:11
33->44:15
17->47:9
8->26:13
18->19:14
3->40:12
46->47:19
41->47:16
13->31:18
1->18:8
32->41:2
32->38:13
11->16:18
27->49:3
44->49:9
4->41:16
13->15:17
39->41:2
10->32:9
37->39:16
3->16:4
8->28:2
4->23:20
41->48:16
18->28:15
14->34:12
18->44:18
7->8:12
31->41:11
5->49:9
0->26:16
38->42:16
0->1:7
1->3:18
3->4:15
4->5:19
5->7:10
7->10:17
10->11:8
11->13:8
13->14:12
14->17:12
17->24:16
24->25:14
25->36:18
36->45:16
45->46:15""")
g.longest_path(0, 49)

(222, 0->1->3->4->5->7->10->11->13->14->17->24->25->36->41->48->49)